In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
%cd /content/drive/MyDrive/Colab Notebooks/Steganography

/content/drive/.shortcut-targets-by-id/1oaZCwXh2SHDAQ3FtWzvS1pKkn3EYFRo1/Colab Notebooks/Steganography


<h1>Text Steganography</h1>

In [ ]:

import sys
from PIL import Image
import random
import binascii

DIST = 8


def normalize_pixel(r, g, b):
 
    if is_modify_pixel(r, g, b):
        seed = random.randint(1, 3)
        if seed == 1:
            r = _normalize(r)
        if seed == 2:
            g = _normalize(g)
        if seed == 3:
            b = _normalize(b)
    return r, g, b


def modify_pixel(r, g, b):
  
    return map(_modify, [r, g, b])


def is_modify_pixel(r, g, b):
   
    return r % DIST == g % DIST == b % DIST == 1


def _modify(i):
    if i >= 128:
        for x in range(DIST + 1):
            if i % DIST == 1:
                return i
            i -= 1
    else:
        for x in range(DIST + 1):
            if i % DIST == 1:
                return i
            i += 1
    raise ValueError


def _normalize(i):
    if i >= 128:
        i -= 1
    else:
        i += 1
    return i


def normalize(path, output):
  
    img = Image.open(path)
    img = img.convert('RGB')
    size = img.size
    new_img = Image.new('RGB', size)

    for y in range(img.size[1]):
        for x in range(img.size[0]):
            r, g, b = img.getpixel((x, y))
            _r, _g, _b = normalize_pixel(r, g, b)
            new_img.putpixel((x, y), (_r, _g, _b))
    new_img.save(output, 'PNG', optimize=True)


def hide_text(path, text):

    text = str(text)

    # convert text to hex for write
    write_param = []
    _base = 0
    for _ in to_hex(text):
        write_param.append(int(_, 16) + _base)
        _base += 16

    # hide hex-text to image
    img = Image.open(path)
    counter = 0
    for y in range(img.size[1]):
        for x in range(img.size[0]):
            if counter in write_param:
                r, g, b = img.getpixel((x, y))
                r, g, b = modify_pixel(r, g, b)
                img.putpixel((x, y), (r, g, b))
            counter += 1

    # save
    img.save(path,'PNG', optimize=True)

def to_hex(s):
    return binascii.hexlify(s.encode()).decode()

def to_str(s):
    return binascii.unhexlify(s).decode()

def read_text(path):
 
    img = Image.open(path)
    counter = 0
    result = []
    for y in range(img.size[1]):
        for x in range(img.size[0]):
            r, g, b = img.getpixel((x, y))
            if is_modify_pixel(r, g, b):
                result.append(counter)
            counter += 1
            if counter == 16:
                counter = 0
    return to_str(''.join([hex(_)[-1:] for _ in result]))


class Text_Steganography(object):
    @classmethod
    def encode(cls, input_image_path, output_image_path, encode_text):
        
        normalize(input_image_path, output_image_path)
        hide_text(output_image_path, encode_text)
        assert read_text(output_image_path) == encode_text, read_text(output_image_path)

    @classmethod
    def decode(cls, image_path):
        
        return read_text(image_path)

<h1>Encryption, hiding and decryption</h1>

In [ ]:
!pip install pycryptodomex

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.1/2.1 MB 23.3 MB/s eta 0:00:00


In [ ]:
from base64 import b64encode, b64decode
import hashlib
from Cryptodome.Cipher import AES
import os
from Cryptodome.Random import get_random_bytes

In [ ]:
#definition of all the functions 

def tostring(lst):
    return ','.join(str(x) for x in lst)

def tolist(str):
    return str.split(',')
    
# define encrypt
def encrypt(plain_text, password):
    # generate a random salt
    salt = get_random_bytes(AES.block_size)

    # use the Scrypt KDF to get a private key from the password
    private_key = hashlib.scrypt(
        password.encode(), salt=salt, n=2**14, r=8, p=1, dklen=32)

    # create cipher config
    cipher_config = AES.new(private_key, AES.MODE_GCM)

    # return a dictionary with the encrypted text
    cipher_text, tag = cipher_config.encrypt_and_digest(bytes(plain_text, 'utf-8'))
    return [
        b64encode(cipher_text).decode('utf-8'),
        b64encode(salt).decode('utf-8'),
        b64encode(cipher_config.nonce).decode('utf-8'),
        b64encode(tag).decode('utf-8')
    ]

# define decrypt
def decrypt(enc_list, password):
    # decode the dictionary entries from base64
    salt = b64decode(enc_list[1])
    cipher_text = b64decode(enc_list[0])
    nonce = b64decode(enc_list[2])
    tag = b64decode(enc_list[3])
    

    # generate the private key from the password and salt
    private_key = hashlib.scrypt(
        password.encode(), salt=salt, n=2**14, r=8, p=1, dklen=32)

    # create the cipher config
    cipher = AES.new(private_key, AES.MODE_GCM, nonce=nonce)

    # decrypt the cipher text
    decrypted = cipher.decrypt_and_verify(cipher_text, tag)

    return decrypted

In [ ]:
meta = [100, 233, 455, 56, 78, 9, 40, 60]
metastr = tostring(meta)

In [ ]:
password = input("Password: ")
encrypted = encrypt(metastr, password)
text_to_hide = tostring(encrypted)

Password: manohar


In [ ]:
text_to_hide

'v92sI0gIzZgFxbdriVeQAynWRXcrrNbmLg==,uwiICp0uh8uSEubMdfT8rA==,bSA0CoAsShFPWWPGXLYsOw==,Lxz6x1I1RfntOAgM+EYimQ=='

In [ ]:
Text_Steganography.encode('Cyan.png','Text_hidden_inside_Cyan.png',text_to_hide)

In [ ]:
extracted = Text_Steganography.decode('Text_hidden_inside_Cyan.png')
print(extracted)

v92sI0gIzZgFxbdriVeQAynWRXcrrNbmLg==,uwiICp0uh8uSEubMdfT8rA==,bSA0CoAsShFPWWPGXLYsOw==,Lxz6x1I1RfntOAgM+EYimQ==


In [ ]:
#decryption:

#converting the extracted text back to list
encrypted = tolist(extracted)

#input password
decrypt_password = input('enter password: ')

#decrypt
try:
    decrypted = decrypt(encrypted, decrypt_password)
    metastr_obt = bytes.decode(decrypted)
except:
    print("Incorrect Password!!")
    metastr_obt = '0'


enter password: manohar


In [ ]:
meta_obt = tolist(metastr_obt)
meta_obt = [int(x) for x in meta_obt]

In [ ]:
meta_obt

[100, 233, 455, 56, 78, 9, 40, 60]